In [43]:
import fastf1
from datetime import datetime
import pandas as pd
import numpy as np
import json
import os
from fastf1.ergast import Ergast
import requests

In [12]:
fastf1.Cache.enable_cache('cache')

In [13]:
def to_native(obj):
    """Convert NumPy/Pandas types to native Python for JSON serialization."""
    if isinstance(obj, (np.integer,)):
        return int(obj)
    elif isinstance(obj, (np.floating,)):
        return float(obj)
    elif isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    return obj

In [14]:
def safe_int(val):
    try:
        if pd.isna(val):
            return None
        return int(val)
    except (ValueError, TypeError):
        return None

In [ ]:
def safe_float(val):
    return None if pd.isna(val) else float(val)

In [45]:
import requests

try:
    r = requests.get('https://ergast.com/api/f1/2023/driverStandings.json')
    r.raise_for_status()
    print("Success! Ergast API reachable.")
except Exception as e:
    print("Failed to connect:", e)


Failed to connect: HTTPSConnectionPool(host='ergast.com', port=443): Max retries exceeded with url: /api/f1/2023/driverStandings.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F3E8AE5A90>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [46]:
def collect_gp_data(year, grand_prix, session_type="R"):
    session = fastf1.get_session(year, grand_prix, session_type)
    session.load()

    gp_data = {
        "year": year,
        "grand_prix": grand_prix,
        "session": session_type,
        "drivers": []
    }

    for driver in session.drivers:
        drv = session.get_driver(driver)
        laps = session.laps.pick_driver(driver)
        tel = session.car_data.get(driver, None)

        driver_data = {
            "driverId": to_native(drv["DriverNumber"]),
            "teamId": to_native(drv["TeamName"]),
            "timestamp": datetime.utcnow().isoformat(),

            # --- Prerace features ---
            "prerace": {
                "driverChampPoints": None,  # External source
                "constructorChampPoints": None,  # External source
                "trackType": None,  # External source
                "expectedWeather": None  # session.weather_data if available
            },

            # --- Qualifying features ---
            "qualifying": {
                "gridPosition": safe_int(session.results.loc[driver, "GridPosition"]) if driver in session.results.index else None,
                "best3LapAvg": safe_float(
                    laps.nsmallest(3, "LapTime")["LapTime"].mean().total_seconds()
                ) if not laps.empty else None,
                "deltaToPole": None,  # Calculate from best lap
                "tireBestLap": None   # Compound from best lap
            },

            # --- Past laps features ---
            "pastLaps": {
                "currentLap": safe_int(laps.iloc[-1]["LapNumber"]) if not laps.empty else None,
                "position": safe_int(laps.iloc[-1]["Position"]) if not laps.empty else None,
                "avgLapTime": safe_float(laps["LapTime"].mean().total_seconds()) if not laps.empty else None,
                "numPitStops": safe_int(laps["PitInTime"].count()) if "PitInTime" in laps else None,
                "timeLostInPits": None,  # Calculate if needed
                "positionsGainedLost": None,
                "deltaToLeader": None,
                "gapAhead": None,
                "gapBehind": None
            },

            # --- Telemetry features ---
            "telemetry": {
                "avgSpeed": safe_float(tel["Speed"].mean()) if tel is not None and "Speed" in tel else None,
                "throttleAvg": safe_float(tel["Throttle"].mean()) if tel is not None and "Throttle" in tel else None,
                "brakeUsagePct": None,  # Needs calculation
                "drsUsageFreq": safe_int(tel["DRS"].sum()) if tel is not None and "DRS" in tel else None,
                "gearShiftsPerLap": None  # Needs calculation
            }
        }

        # Convert everything to native types
        driver_data = json.loads(json.dumps(driver_data, default=to_native))
        gp_data["drivers"].append(driver_data)

    return gp_data

In [49]:
def default_converter(o):
    if isinstance(o, (pd.Series, pd.DataFrame)):
        return o.to_dict()
    if hasattr(o, "item"):
        return o.item()
    return str(o)


In [50]:
def save_gp_data_for_year(year):
    schedule = fastf1.get_event_schedule(year)
    for _, race in schedule.iterrows():
        grand_prix = race['EventName']
        print(f"Processing {year} {grand_prix} ...")
        
        try:
            data = collect_gp_data(year, grand_prix)
        except Exception as e:
            print(f"Failed to collect data for {grand_prix} {year}: {e}")
            continue
        
        # Create folder per year
        folder = f"datasets/{year}"
        os.makedirs(folder, exist_ok=True)
        
        # Clean GP name for filename
        gp_clean = grand_prix.replace(' ', '_').replace('/', '_')
        file_path = os.path.join(folder, f"{year}_{gp_clean}_full.json")

        with open(file_path, "w") as f:
            json.dump(data, f, indent=2, default=default_converter)

        print(f"Saved {year} {grand_prix} data to {file_path}")


In [ ]:
if __name__ == "__main__":
    for y in [2023, 2024]:
        save_gp_data_for_year(y)


events      WARNING 	Correcting user input 'Pre-Season Testing' to 'British Grand Prix'
core           INFO 	Loading data for British Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


Processing 2023 Pre-Season Testing ...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Saved 2023 Pre-Season Testing data to datasets/2023\2023_Pre-Season_Testing_full.json
Processing 2023 Bahrain Grand Prix ...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

Saved 2024 Las Vegas Grand Prix data to datasets/2024\2024_Las_Vegas_Grand_Prix_full.json
Processing 2024 Qatar Grand Prix ...


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNIN

Saved 2024 Qatar Grand Prix data to datasets/2024\2024_Qatar_Grand_Prix_full.json
Processing 2024 Abu Dhabi Grand Prix ...


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNIN

Saved 2024 Abu Dhabi Grand Prix data to datasets/2024\2024_Abu_Dhabi_Grand_Prix_full.json
